In [1]:
import sys
import torch
import numpy as np
from torch.utils.data import DataLoader
import torch.nn as nn
from torchvision import transforms, datasets
import pytorch_lightning as pl
from tqdm.notebook import trange, tqdm
import matplotlib.pyplot as plt

sys.path.insert(1, 'C:\\Users\\ruchi\\Desktop\\Research\\TLab\\HyperDimensional\\Tools')

import HDComputing as hdc

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

C:\Users\ruchi\Anaconda3\envs\EE510\lib\site-packages\pytorch_lightning\utilities\distributed.py:37: UserWarning: Unsupported `ReduceOp` for distributed computing.
  warnings.warn(*args, **kwargs)


In [2]:
class HDCodec(nn.Module):
	def __init__(self):
		super().__init__()

		enc = {'record': {'N': 400, 'M': 50, 'range': (-0.25, 0.25)}}
		self.space = hdc.Hyperspace(rep=hdc.BSCVector, dim=10000, enc=enc)

	def forward(self, x, batch_labels):
		feature_list = x.detach().cpu().numpy()

		for num in range(100):
			label = str(batch_labels[num].item())
			features = feature_list[num]

			self.space.add(name=label, features=features)

		return x


class CNN_HD(pl.LightningModule):

	def __init__(self):
		super().__init__()
		self.convs = nn.Sequential(nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1, padding=2, bias=True),
		                           nn.ReLU(),
		                           nn.MaxPool2d(kernel_size=2),
		                           nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1, padding=0, bias=True),
		                           nn.ReLU(),
		                           nn.MaxPool2d(kernel_size=2, stride=2))
		self.fc = nn.Sequential(nn.Linear(16*5*5, 120),
		                        nn.ReLU(),
								nn.Linear(120, 84),
		                        nn.ReLU(),
								nn.Linear(84, 10))
		self.codec = HDCodec()
		self.loss_fn = nn.CrossEntropyLoss()

	def forward(self, img, label, epoch_idx):
		x = self.convs(img)
		x = x.view(-1, 16*5*5)
		x = self.codec(x, label, epoch_idx)
		x = self.fc(x)
		return x

	def configure_optimizers(self):
		optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
		return optimizer

	def training_step(self, batch, batch_idx):
		imgs, labels = batch
		predictions = self(imgs, labels)
		loss = self.loss_fn(predictions, labels)
		result = pl.TrainResult(loss)
		result.log('train_loss', loss, prog_bar=True)
		return result

In [3]:
train_dataset = datasets.MNIST('../data', train=True, download=False,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ]))

test_dataset = datasets.MNIST('../data', train=False, download=False,
                        transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ]))

train_loader = DataLoader(train_dataset, batch_size=100, shuffle=True, num_workers=8)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True)

model = CNN_HD()
trainer = pl.Trainer(gpus=1, benchmark=True, max_epochs=10)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]


In [4]:
trainer.fit(model, train_loader)

C:\Users\ruchi\Anaconda3\envs\EE510\lib\site-packages\pytorch_lightning\utilities\distributed.py:37: UserWarning: Could not log computational graph since the `model.example_input_array` attribute is not set or `input_array` was not given
  warnings.warn(*args, **kwargs)

  | Name    | Type             | Params
---------------------------------------------
0 | convs   | Sequential       | 2 K   
1 | fc      | Sequential       | 59 K  
2 | codec   | HDCodec          | 0     
3 | loss_fn | CrossEntropyLoss | 0     
Saving latest checkpoint..


1